<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/AI_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI')

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import copy
import json
from openai import OpenAI
from openai import ChatCompletion

#from utils_local import o1_tools

client = OpenAI(api_key=openai_api_key)
#O1_MODEL = 'o1-mini'
GPT_MODEL = 'gpt-4o-mini'

In [4]:
# System prompt for the execution model
gpt4o_system_prompt = """
You are a helpful data analyst responsible for analysis provided data file using the tools provided.
You must explain your decision-making process across various steps. if ask to read data, you should read the
data file and using the provided tool and summurize the infomation in in the first 5 rows.  If you are not asked to read
data, you should answer a user question to the best of your ability.

"""

In [5]:
tool_metadata = {
    "name": "read_csv_to_dataframe",
    "description": "Reads a CSV file and returns the DataFrame.",
    "parameters": {
        "type": "object",
        "properties": {
            "file_path": {"type": "string", "description": "Path to the CSV file."}
        },
        "required": ["file_path"]
    }
}

In [6]:
tools = [tool_metadata]

In [26]:
tools

[{'name': 'read_csv_to_dataframe',
  'description': 'Reads a CSV file and returns the DataFrame.',
  'parameters': {'type': 'object',
   'properties': {'file_path': {'type': 'string',
     'description': 'Path to the CSV file.'}},
   'required': ['file_path']}}]

In [7]:
# Tools
def read_csv_to_dataframe(file_path: str) -> str:
    import pandas as pd
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        return f"Error reading CSV file: {e}"

In [8]:
# Map function names to actual functions
function_mapping = {
    'read_csv_to_dataframe': read_csv_to_dataframe
}

In [28]:
data = '/content/Beer_small_test.csv'
messages = [
    {"role": "system", "content": 'you are helful data analyst reading the data files'},
    {"role": "user", "content": f"Can you read the {data} and summurize first 5 rows"}
]
# User query
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    functions=tools,
    function_call={"name": "read_csv_to_dataframe"}
)

In [29]:
response.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"file_path":"/content/Beer_small_test.csv"}', name='read_csv_to_dataframe'), tool_calls=None)

In [ ]:
def call_gpt4o(plan):
    gpt4o_policy_prompt = gpt4o_system_prompt.replace("{policy}", plan)
    messages = [
        {'role': 'system', 'content': gpt4o_policy_prompt},
    ]

    while True:
        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
            tools=TOOLS,
            parallel_tool_calls=False
        )

        assistant_message = response.choices[0].message.to_dict()
        print(assistant_message)
        messages.append(assistant_message)

        append_message({'type': 'assistant', 'content': assistant_message.get('content', '')})

        if (response.choices[0].message.tool_calls and
            response.choices[0].message.tool_calls[0].function.name == 'instructions_complete'):
            break

        if not response.choices[0].message.tool_calls:
            continue

        for tool in response.choices[0].message.tool_calls:
            tool_id = tool.id
            function_name = tool.function.name
            input_arguments_str = tool.function.arguments

            append_message({'type': 'tool_call', 'function_name': function_name, 'arguments': input_arguments_str})

            try:
                input_arguments = json.loads(input_arguments_str)
            except (ValueError, json.JSONDecodeError):
                continue

            if function_name in function_mapping:
                try:
                    function_response = function_mapping[function_name](**input_arguments)
                except Exception as e:
                    function_response = {'error': str(e)}
            else:
                function_response = {'error': f"Function '{function_name}' not implemented."}

            try:
                serialized_output = json.dumps(function_response)
            except (TypeError, ValueError):
                serialized_output = str(function_response)

            messages.append({
                "role": "tool",
                "tool_call_id": tool_id,
                "content": serialized_output
            })

            append_message({'type': 'tool_response', 'function_name': function_name, 'response': serialized_output})

    return messages

In [30]:
#test block 1

import json
import pandas as pd

# Define the tool (function metadata)
tools = [
    {
        "name": "read_csv_to_dataframe",
        "description": "Reads a CSV file and converts it into a pandas DataFrame.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {"type": "string", "description": "Path to the CSV file to read."}
            },
            "required": ["file_path"]
        }
    }
]

# Define the function to read the CSV
def read_csv_to_dataframe(file_path: str):
    try:
        df = pd.read_csv(file_path)
        return {
            "rows": len(df),
            "columns": list(df.columns),
            "head": df.head(3).to_dict(orient="records")
        }
    except Exception as e:
        return {"error": str(e)}

# Define the messages for the assistant
messages = [
    {"role": "system", "content": "You are an assistant that can process CSV files."},
    {"role": "user", "content": "Can you read the file /content/Beer_small_test.csv and tell me about it?"}
]

# Simulate the GPT-4o response (mock client)
class MockGPT4oClient:
    def chat(self, completions):
        # Simulate what the GPT-4o would return
        return {
            "choices": [
                {
                    "message": {
                        "function_call": {
                            "name": "read_csv_to_dataframe",
                            "arguments": json.dumps({"file_path": "/content/Beer_small_test.csv"})
                        }
                    }
                }
            ]
        }

# Instantiate the client
client = MockGPT4oClient()

# Generate the response
response = client.chat(
    completions={
        "model": "gpt-4o",
        "messages": messages,
        "functions": tools,
        "function_call": {"name": "read_csv_to_dataframe"}
    }
)

# Process the response
if response["choices"][0]["message"].get("function_call"):
    function_call = response["choices"][0]["message"]["function_call"]
    function_name = function_call["name"]
    function_arguments = json.loads(function_call["arguments"])

    if function_name == "read_csv_to_dataframe":
        file_path = function_arguments["file_path"]

        # Call the function and handle the output
        result = read_csv_to_dataframe(file_path)

        # Display the result
        print(f"Function Output: {result}")
else:
    print("No function call found in the response.")

Function Output: {'rows': 781, 'columns': ['string_name', 'string_beer_id', 'string_brewer_id', 'number_abv', 'string_style', 'number_appearance', 'number_aroma', 'number_palate', 'number_taste', 'ratings', 'date_time', 'string_profile_name', 'text'], 'head': [{'string_name': 'Sausa Weizen', 'string_beer_id': 47986, 'string_brewer_id': 10325, 'number_abv': 5.0, 'string_style': 'Hefeweizen', 'number_appearance': 2.5, 'number_aroma': 2.0, 'number_palate': 1.5, 'number_taste': 1.5, 'ratings': 1.5, 'date_time': '2009-02-16T15:57:03Z', 'string_profile_name': 'stcules', 'text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.'}, {'string_name': 'Red Moon', 'string_beer_id': 48213, 'string_brewer_id': 10325, 'number_abv': 6.2, 'string_style': 'English Strong Ale', 'number_appea

In [31]:
response

{'choices': [{'message': {'function_call': {'name': 'read_csv_to_dataframe',
     'arguments': '{"file_path": "/content/Beer_small_test.csv"}'}}}]}

In [ ]:
#test block 2
import json
import pandas as pd

# Define the tool for reading the CSV
tools = [
    {
        "name": "read_csv_to_dataframe",
        "description": "Reads a CSV file and extracts the requested column data.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {"type": "string", "description": "Path to the CSV file to read."},
                "column_name": {"type": "string", "description": "Name of the column to extract."},
                "num_entries": {"type": "integer", "description": "Number of entries to retrieve from the column."}
            },
            "required": ["file_path", "column_name", "num_entries"]
        }
    }
]

# Define the function to read the CSV and extract data
def read_csv_to_dataframe(file_path: str, column_name: str, num_entries: int):
    try:
        df = pd.read_csv(file_path)
        if column_name not in df.columns:
            return {"error": f"Column '{column_name}' not found in the file."}
        return {
            "data": df[column_name].head(num_entries).tolist(),
            "column_name": column_name
        }
    except Exception as e:
        return {"error": str(e)}

# Define the initial query
messages = [
    {"role": "system", "content": "You are an assistant that can process CSV files and analyze data."},
    {"role": "user", "content": "Please summarize the first 5 entries in the 'text' column of the file '/content/Beer_small_test.csv'."}
]

# Simulate the GPT-4o response (mock client)
class MockGPT4oClient:
    def chat(self, completions):
        # Simulate the GPT-4o model suggesting a function call
        return {
            "choices": [
                {
                    "message": {
                        "function_call": {
                            "name": "read_csv_to_dataframe",
                            "arguments": json.dumps({
                                "file_path": "/content/Beer_small_test.csv",
                                "column_name": "text",
                                "num_entries": 5
                            })
                        }
                    }
                }
            ]
        }

# Instantiate the client
client = MockGPT4oClient()

# Generate the response
response = client.chat(
    completions={
        "model": "gpt-4o",
        "messages": messages,
        "functions": tools,
        "function_call": {"name": "read_csv_to_dataframe"}
    }
)

# Process the response
if response["choices"][0]["message"].get("function_call"):
    function_call = response["choices"][0]["message"]["function_call"]
    function_name = function_call["name"]
    function_arguments = json.loads(function_call["arguments"])

    if function_name == "read_csv_to_dataframe":
        # Call the function
        result = read_csv_to_dataframe(
            file_path=function_arguments["file_path"],
            column_name=function_arguments["column_name"],
            num_entries=function_arguments["num_entries"]
        )

        # Check for errors
        if "error" in result:
            user_answer = f"I tried to read the file, but encountered an error: {result['error']}"
        else:
            # Extracted data
            extracted_data = result["data"]
            column_name = result["column_name"]

            # Craft a new prompt for summarization
            new_messages = [
                {"role": "system", "content": "You are an assistant that analyzes text data."},
                {"role": "user", "content": f"Please summarize the following data from the '{column_name}' column: {extracted_data}"}
            ]

            # Simulate the model summarizing the text
            summary_response = client.chat(
                completions={
                    "model": "gpt-4o",
                    "messages": new_messages
                }
            )

            # Mock summary response
            summary = (
                "The extracted entries discuss various aspects, including the flavor, texture, and uniqueness "
                "of different beers. Common themes include customer preferences and the distinct character of each beer."
            )

            user_answer = f"Summary of the '{column_name}' column:\n{summary}"

        # Print the response
        print(user_answer)
else:
    print("No function call found in the response.")